In [21]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import os
import time

# 크롬드라이버 경로 (현재 파이썬 파일과 동일한 디렉토리)
chromedriver_path = os.path.join(os.getcwd(), 'chromedriver.exe')

# 셀레니움 옵션 설정
options = Options()
options.add_argument('--headless')
options.add_argument('--disable-gpu')

# 드라이버 로드
service = Service(executable_path=chromedriver_path)
driver = webdriver.Chrome(service=service, options=options)

# 타겟 URL 접속
url = "https://auto.danawa.com/auto/?Work=record&Tab=Model&Classify=PS~&Month=2023-01-00&MonthTo="
driver.get(url)

# JS 로딩 대기
time.sleep(3)

# 페이지 소스 파싱
soup = BeautifulSoup(driver.page_source, 'html.parser')
car_tags = soup.select('#autodanawa_gridC > div.gridMain > article > main > div > table > tbody > tr> td.title > a')

# 결과 출력
for tag in car_tags:
    print(tag.text.strip())

# 드라이버 종료
driver.quit()


Panamera
Taycan
911
AMG GT
M4
718 Boxster
718 Cayman
Mustang
GR SUPRA
M3
M5
GR 86
F-Type
Z4


In [33]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import os
import time
import pandas as pd

# 크롬드라이버 경로 설정 (현재 디렉토리 기준)
chromedriver_path = os.path.join(os.getcwd(), 'chromedriver.exe')

# 셀레니움 옵션 설정
options = Options()
options.add_argument('--headless')  # 창 없이 실행
options.add_argument('--disable-gpu')

# 드라이버 실행
service = Service(executable_path=chromedriver_path)
driver = webdriver.Chrome(service=service, options=options)

# 타겟 URL
url = "https://auto.danawa.com/auto/?Work=record&Tab=Model&Classify=PS~&Month=2023-01-00&MonthTo="
driver.get(url)
time.sleep(3)  # 페이지 로딩 대기

# 페이지 소스 가져오기
soup = BeautifulSoup(driver.page_source, 'html.parser')

# 각 요소 셀렉터로 추출
model_tags = soup.select('#autodanawa_gridC > div.gridMain > article > main > div > table > tbody > tr > td.title > a')
sales_tags = soup.select('#autodanawa_gridC > div.gridMain > article > main > div > table > tbody > tr > td.num')
rate_tags = soup.select('#autodanawa_gridC > div.gridMain > article > main > div > table > tbody > tr > td.rate.right')
prev_tags = soup.select('#autodanawa_gridC > div.gridMain > article > main > div > table > tbody > tr > td:nth-child(7)')

# 판매량 텍스트 정제 (button 제거)
cleaned_sales = []
for tag in sales_tags:
    for button in tag.find_all('button'):
        button.decompose()
    cleaned_sales.append(tag.get_text(strip=True).replace(",", ""))

# 전월대비 텍스트 정제 (줄바꿈 포함)
cleaned_prev = []
for tag in prev_tags:
    base = tag.get_text(strip=True)
    span = tag.find('span')
    diff = span.get_text(strip=True) if span else ""
    if diff and diff in base:
        base = base.replace(diff, "").strip()
    cleaned_prev.append(f"{base}   {diff}")

# 결과 저장
data = []
for model, sales, rate, prev in zip(model_tags, cleaned_sales, rate_tags, cleaned_prev):
    data.append({
        "자동차 모델": model.text.strip(),
        "판매량": sales,
        "점유율": rate.text.strip(),
        "전월대비": prev
    })

# 데이터프레임으로 변환
df = pd.DataFrame(data)

# 출력 확인
print(df)

# 드라이버 종료
driver.quit()


         자동차 모델  판매량    점유율       전월대비
0      Panamera  124  33.2%  142   18▼
1        Taycan   58  17.4%    73   8▼
2           911   35  16.6%   34   28▲
3        AMG GT   10   8.8%  118   85▼
4            M4   15   5.6%   31   10▼
5   718 Boxster    3   4.0%    21   6▼
6    718 Cayman    3   3.2%    20   8▼
7       Mustang   65   2.9%   26   15▼
8      GR SUPRA   39   2.4%     0   9▲
9            M3    1   2.1%   25   17▼
10           M5    4   1.6%   23   17▼
11        GR 86    2   1.1%     8   4▼
12           Z4   13   0.5%    11   9▼
13       F-Type    1   0.5%     0   2▲


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import os
import time
import pandas as pd

# 크롬드라이버 경로 설정
chromedriver_path = os.path.join(os.getcwd(), 'chromedriver.exe')

# 셀레니움 옵션 설정
options = Options()
options.add_argument('--headless')  # 창 없이 실행
options.add_argument('--disable-gpu')

# 드라이버 실행
service = Service(executable_path=chromedriver_path)
driver = webdriver.Chrome(service=service, options=options)

# 전체 데이터를 저장할 리스트
all_data = []

# 연도, 월 반복
for year in range(2023, 2026):
    for month in range(1, 13):
        if year == 2025 and month > 3:
            break

        url = f"https://auto.danawa.com/auto/?Work=record&Tab=Model&Classify=PS~&Month={year}-{month:02d}-00&MonthTo="
        print(f"크롤링 중: {url}")
        driver.get(url)
        time.sleep(3)

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        model_tags = soup.select('#autodanawa_gridC > div.gridMain > article > main > div > table > tbody > tr > td.title > a')
        sales_tags = soup.select('#autodanawa_gridC > div.gridMain > article > main > div > table > tbody > tr > td.num')
        rate_tags = soup.select('#autodanawa_gridC > div.gridMain > article > main > div > table > tbody > tr > td.rate.right')
        prev_tags = soup.select('#autodanawa_gridC > div.gridMain > article > main > div > table > tbody > tr > td:nth-child(7)')

        data = []
        for i in range(len(model_tags)):
            model = model_tags[i].text.strip()

            # 판매량 정제
            sales_raw = sales_tags[i]
            for button in sales_raw.select('button'):
                button.extract()
            cleaned_sales = sales_raw.text.strip().replace(",", "")

            rate = rate_tags[i].text.strip()

            # 전월대비 정제 (기준값과 증감값 분리)
            prev_tag = prev_tags[i]
            change_span = prev_tag.select_one('span')

            if change_span:
                change_class = change_span.get('class')[0]  # "up" 또는 "down"
                change_number = change_span.text.strip()
                if 'down' in change_class:
                    change_value = f"-{change_number}"
                elif 'up' in change_class:
                    change_value = f"+{change_number}"
                else:
                    change_value = change_number
                change_span.extract()  # 기준값만 남기기 위해 제거
            else:
                change_value = ''

            prev_value = prev_tag.text.strip()

            data.append({
                "년도": year,
                "월": month,
                "자동차 모델": model,
                "판매량": cleaned_sales,
                "점유율": rate,
                "전월대비_기준": prev_value,
                "전월대비_판매증감": change_value
            })

        df_month = pd.DataFrame(data)
        all_data.append(df_month)

# 전체 데이터프레임 통합
final_df = pd.concat(all_data, ignore_index=True)

# CSV 저장
final_df.to_csv('car_sales_2023_01_to_2025_03.csv', index=False, encoding='utf-8-sig')
print("CSV 파일 저장 완료")

# 드라이버 종료
driver.quit()


크롤링 중: https://auto.danawa.com/auto/?Work=record&Tab=Model&Classify=PS~&Month=2023-01-00&MonthTo=
크롤링 중: https://auto.danawa.com/auto/?Work=record&Tab=Model&Classify=PS~&Month=2023-02-00&MonthTo=
크롤링 중: https://auto.danawa.com/auto/?Work=record&Tab=Model&Classify=PS~&Month=2023-03-00&MonthTo=
크롤링 중: https://auto.danawa.com/auto/?Work=record&Tab=Model&Classify=PS~&Month=2023-04-00&MonthTo=
크롤링 중: https://auto.danawa.com/auto/?Work=record&Tab=Model&Classify=PS~&Month=2023-05-00&MonthTo=
크롤링 중: https://auto.danawa.com/auto/?Work=record&Tab=Model&Classify=PS~&Month=2023-06-00&MonthTo=
크롤링 중: https://auto.danawa.com/auto/?Work=record&Tab=Model&Classify=PS~&Month=2023-07-00&MonthTo=
크롤링 중: https://auto.danawa.com/auto/?Work=record&Tab=Model&Classify=PS~&Month=2023-08-00&MonthTo=
크롤링 중: https://auto.danawa.com/auto/?Work=record&Tab=Model&Classify=PS~&Month=2023-09-00&MonthTo=
크롤링 중: https://auto.danawa.com/auto/?Work=record&Tab=Model&Classify=PS~&Month=2023-10-00&MonthTo=
크롤링 중: https://auto.